# Lab 1: Information Retrieval

__Students:__ mindu931, karzi360

 ### Overall idea:
 1. Crawling webpage of overview to get source code of overview webpage (=content)
 2. From this long source code information from source code of overview page, get all the 
 available linked apps (since identifier = url of the link = url of the app detail page itself,
 finding all the identifiers following one specific pattern is sufficient) (=appreg = regular
 expression for finding the app-url identfiers; app_url_list = (unique) list of the detected
 app-urls satisfying the pattern)
 3. Now accessing each app-url individually and looking in the corresponding detail page
 source code for the description of the app --> accessable by another pattern (=desc_container)
 4. All description information + additional pattern information is stored, but needs to be
 cleaned for data preprocessing (<html>... information not important for text mining)

### Crawling



a) Get the webpage content by using functions in 
__[urllib module](https://docs.python.org/3/library/urllib.html#module-urllib)__.

b) Get app url by regular expression using functions from __[re module](https://docs.python.org/3/library/re.html?highlight=re#module-re)__.

The code block below will run both a and b part

In [3]:
import urllib.request
import re

appreg = r'href=\"(/store/apps/details.*?)\"'
appre = re.compile(appreg)

all_list = []
url = 'https://play.google.com/store/apps/category/GAME?hl=en'
content = urllib.request.urlopen(url).read().decode('utf-8')
app_url_list = re.findall(appre,content)
app_url_list = list(set(app_url_list))
#print(app_url_list)
#print(len(app_url_list))
all_list.extend(app_url_list)
        
print(len(all_list))
#print(all_list)

70


c) Access specific webpage to get description of each app and then store the description in files.

Now, we will go throught each url to get the description and app name. We create a file for each app, with tittle of the file is the name of the app. List of all apps is saved in the "files" list

In [4]:
def cleanHTML(html):
    html = re.sub('<.*?>', ' ', html) #remove the tags
    html = re.sub('&.*?;', ' ', html) #remove the html symbols (ex: &amp;)
    html = re.sub('https?:\/\/.*?(\s|\'|$)', ' ', html) #remove urls
    html = re.sub('[^\w]', ' ', html) #remove non alpha-numeric or space characters
    return html

def get_and_clean(phrase, page):
    content = re.findall(phrase ,page)
    content = ''.join(content)
    content = cleanHTML(content)
    content = content.replace('itemprop  description', '')
    return(content)

#List files
files = []
print('Here is the list of apps that crawled')

#Check the number of apps crawled
while len(files) < 1000 :
    url = all_list[0]
    all_list = all_list[1:]
    
    # phrase for getting name and description
    content_reg = "itemprop=\"description.*?\">.*?<div jsname=\".*?\">.*?</div>"
    nameg = r'<title id=\"main-title\">(.*?) - Apps'
    name_reg = re.compile(nameg)
    
    page = urllib.request.urlopen('https://play.google.com' + url +"&hl=en").read().decode('utf8')
    
    
    # only proceeding, when checked page contained some name, 
    # otherwise proceed with next app on all_list
    if re.search(name_reg, page) != None :       
            app_name = get_and_clean(name_reg, page)
            #print(app_name)

            if app_name not in files:
                files.append(app_name)
                
                # for checking loop but only after every 50th iteration
                description = get_and_clean(content_reg, page)
                if len(files) % 50 == 0:
                            print("Number of already collected descriptions: {}".format(len(files)))
                        
                temp = re.findall(appre, page) #getting new app names for next step
                all_list = list(set(all_list + temp)) # creating new app_url_list for 
                
                #Write the file
                try:
                    with open(app_name , 'w') as file:
                            print(app_name)
                            #print(description)
                            file.write(description)
                            file.close()
                except:
                    print(app_name) # can't write the file (character issue)



Here is the list of apps that crawled
Dr  Driving
Block Craft 3D  Building Simulator Games For Free
Chess
Puzzle Game
Subway Surfers
Pou
Farming Simulator 18
Dedi App 17
Traffic Rider
Beach Buggy Racing
True Skate
Stickman Skate Battle
City Skater   Rule the Skate Park 
Stickman Soccer 2018
Jet Car Stunts 2
Dr  Driving 2
Bike Race Free   Top Motorcycle Racing Games
Traffic Racer
FIFA Soccer
Merge Plane   Click   Idle Tycoon
Asphalt 8  Airborne
Asphalt Nitro
Real Chess
Real Checkers
Real Checkers
Chess Online
Russian Checkers
Spanish Checkers
English to Hindi Dictionary
English to Hindi Dictionary
Brazilian Checkers
eBook Reader   Your Free audio eBook Library
Checkers
Checkers
Checkers Free
Sudoku
Sudoku
Tic Tac Toe Free
English Hindi Dictionary
English Hindi Dictionary
Dictionary   Merriam Webster
Pocket Thesaurus
Pocket Quotations Premium
Learner s Dictionary   English
WordReference com dictionaries
English French Thesaurus
POC Bible  Malayalam 
Take Off
Jesus Youth Prayer
Jesus Yout

### Construct Inverted file index (Vector Model)



d) Preprocess text using NLP techniques from __[nltk module](http://www.nltk.org/py-modindex.html)__ or spaCy.

Using nltk.download(ID) to get the corpora if it is not downloaded before. __[nltk corpora](http://www.nltk.org/nltk_data/)__

In [1]:
import nltk
nltk.download('stopwords')

stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to C:\Users\Duong Minh
[nltk_data]     Duc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Now, we will clean the descriptions of each app.

In [5]:
# We choose PorterSteammer for stem 
porter = nltk.PorterStemmer()

def pre_process(content):
    #print(content)
    
    #remove non alpha-numeric character
    content = re.sub('[^A-Za-z0-9]+', ' ', content)
    
    #Lowercase
    content = content.lower()

    #We can use tolenize function - same result
    words = nltk.word_tokenize(content)
    
    #Remove stopwords
    meanningful_word = [word for word in words if word not in stop]

    #Stem
    stems = []
    for word in meanningful_word:
        stem = porter.stem(word)
        stems.append(stem)
    #print(stems)

    content = ' '.join(stems)
    return(content)
    

print('Here is the list of apps that cleaned')
for app_name in files:
    #print(app_name)
    with open(app_name, 'r+') as file:
        content = file.read()
        text = (pre_process(content))
        file.truncate()
        file.seek(0)
        print(app_name)
        file.write(text)
        file.close()
        #print(text)

Here is the list of apps that cleaned
Dr  Driving
Block Craft 3D  Building Simulator Games For Free
Chess
Puzzle Game
Subway Surfers
Pou
Farming Simulator 18
Dedi App 17
Traffic Rider
Beach Buggy Racing
True Skate
Stickman Skate Battle
City Skater   Rule the Skate Park 
Stickman Soccer 2018
Jet Car Stunts 2
Dr  Driving 2
Bike Race Free   Top Motorcycle Racing Games
Traffic Racer
FIFA Soccer
Merge Plane   Click   Idle Tycoon
Asphalt 8  Airborne
Asphalt Nitro
Real Chess
Real Checkers
Chess Online
Russian Checkers
Spanish Checkers
English to Hindi Dictionary
Brazilian Checkers
eBook Reader   Your Free audio eBook Library
Checkers
Checkers Free
Sudoku
Tic Tac Toe Free
English Hindi Dictionary
Dictionary   Merriam Webster
Pocket Thesaurus
Pocket Quotations Premium
Learner s Dictionary   English
WordReference com dictionaries
English French Thesaurus
POC Bible  Malayalam 
Take Off
Jesus Youth Prayer
Jesus Youth
Tecarta Bible
Barnabodha
Learn Hindi through English
Learn Hindi through Tamil
Le

...)Compute tdidf 
eg. Using functions from __[scikit-learn module](http://scikit-learn.org/stable/modules/classes.html)__. TfidfVectorizer is used for converting a collection of raw documents to a matrix of TF-IDF features.
#### You can also build the tfidf matrix with other library or your own algorithm

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
transvector = TfidfVectorizer()

corpus = []

# scan all documents and take the text to make a corpus
for file in files: 
    with open(file, 'r') as f:
        corpus.append(f.readline().strip())
#print(corpus)

#Calculate
tfidf1 = transvector.fit_transform(corpus)
tfidf_matrix = tfidf1.toarray()
tfidf_matrix.shape
#print(tfidf_matrix)
#print(transvector.get_feature_names())
#print(transvector)

(100, 4109)

### Query Process

eg. "Dragon, Control, hero, running"

eg. "The hero controls the dragon to run."



In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import operator
import pandas as pd
import numpy as np

#here is the query process function
def query_process(query):
    #clean the query, like clean the descriptions
    query = pre_process(query)
    #print(query)
    query_word = query.split()
    
    #Calculating
    query = transvector.transform(list(query_word)).toarray()
    similarity = cosine_similarity(tfidf_matrix, query)
    #print(similarity)
    
    simi_average = []
    index = 0
    
    for doc in similarity:
        value = sum(doc)/4
        simi_average.append(value)
        
    top3 = sorted(zip(simi_average, files), reverse=True)[:3]

    #print(index)
    result = pd.DataFrame(np.array(top3).reshape(3,2), columns=["Average similarity", "Apps name"])
    return(result)

#query_process('Dragon, Control, hero, running')

In [9]:
query_process('Dragon, Control, hero, running')

,Similarity,Apps name
0,0.09442785765643148,Farm Heroes Saga
1,0.052329419449271726,Rise Up
2,0.03752798900373222,Star Wars KOTOR


In [10]:
query_process('The hero controls the dragon to run.')

,Similarity,Apps name
0,0.09442785765643148,Farm Heroes Saga
1,0.052329419449271726,Rise Up
2,0.03752798900373222,Star Wars KOTOR


In [11]:
query_process('playing music')

,Similarity,Apps name
0,0.05189540879338904,Bike Race Free Top Motorcycle Racing Games
1,0.04154004047104521,Chess Online
2,0.040371236586411424,8 Ball Pool
